In [ ]:
#İndirilmesi gereken kütüphaneler
!pip install simpletransformers

In [ ]:
#Data setini yüklemek için drive'ı içe aktarma
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import nltk
import json

# NLTK paketini indir (Gerekli NLTK kaynaklarını indirir)
nltk.download('punkt')

# CSV dosyasını oku (Veri setini CSV dosyasından okur)
df = pd.read_csv("/content/sonHali.csv")

# Yalnızca 'text', 'kullanım_label' ve 'hizmet_label' sütunlarını kullan (Gereksiz sütunları çıkarır)
df = df[['text', 'kullanım_label', 'hizmet_label']]

# JSON formatında veriyi hazırlamak (Veriyi JSON formatına uygun hale getirir)
data = {
    "Text": df['text'].tolist(),  # Tüm metinleri liste olarak ekler
    "Entities": []  # Her bir metin için etiketli varlıkları tutacak boş bir liste oluşturur
}

# Her satırdaki veriyi işler
for index, row in df.iterrows():
    entity_list = []  # Bu satır için varlıkları tutacak liste

    # kullanım_label sütunundaki etiketleri ekle
    if not pd.isna(row['kullanım_label']):
        kullanım_entities = json.loads(row['kullanım_label'])  # JSON stringini sözlüğe çevirir
        for entity in kullanım_entities:
            entity_list.append({
                "start": entity["start"],  # Varlık başlangıç indexi
                "end": entity["end"],  # Varlık bitiş indexi
                "text": entity["text"],  # Varlığın metni
                "labels": entity["labels"]  # Varlığın etiketleri
            })

    # hizmet_label sütunundaki etiketleri ekle
    if not pd.isna(row['hizmet_label']):
        hizmet_entities = json.loads(row['hizmet_label'])  # JSON stringini sözlüğe çevirir
        for entity in hizmet_entities:
            entity_list.append({
                "start": entity["start"],  # Varlık başlangıç indexi
                "end": entity["end"],  # Varlık bitiş indexi
                "text": entity["text"],  # Varlığın metni
                "labels": entity["labels"]  # Varlığın etiketleri
            })

    # Tüm varlıkları bu metin için listeye ekler
    data["Entities"].append(entity_list)

# JSON dosyasını kaydet (Veriyi JSON dosyasına yazar)
output_json_path = "output_son.json"
with open(output_json_path, "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print("JSON dosyası başarıyla oluşturuldu.")

# JSON dosyasını aç ve içeriği bir değişkene ata (JSON dosyasını okur ve veriyi yükler)
with open(output_json_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Veriyi bir DataFrame'e yükle (JSON verisini pandas DataFrame formatına çevirir)
df = pd.DataFrame({
    "Text": data["Text"],  # Metin sütunu
    "Entities": data["Entities"]  # Varlıklar sütunu
})

# Metinleri tokenize edip varlıkları etiketlemek için bir fonksiyon
def tokenize_and_annotate(df):
    tokens = []  # Token'ları tutacak liste
    labels = []  # Etiketleri tutacak liste
    for _, row in df.iterrows():
        tweet = row['Text']
        entities = row['Entities']

        # Metnin string formatında olduğundan emin olun
        if not isinstance(tweet, str):
            tweet = str(tweet)

        # Metni tokenize et
        tokenized_text = nltk.word_tokenize(tweet)
        token_labels = ['O'] * len(tokenized_text)  # Tüm token'lar için varsayılan etiket 'O'

        # Her varlık için
        for entity in entities:
            start_idx = entity['start']  # Varlık başlangıç indeksi
            end_idx = entity['end']  # Varlık bitiş indeksi
            entity_text = entity['labels'][0]  # İlk etiketi alır (Varsayılan olarak bir etiket olduğu varsayılır)

            # Varlık için token indekslerini bul
            char_idx = 0
            token_start = -1
            token_end = -1
            for i, token in enumerate(tokenized_text):
                char_idx = tweet.find(token, char_idx)
                if char_idx == start_idx:
                    token_start = i
                if char_idx + len(token) >= end_idx:
                    token_end = i
                    break
                char_idx += len(token)

            # Eğer varlık bulunmuşsa, etiketleri atar
            if token_start != -1 and token_end != -1:
                token_labels[token_start] = f'B-{entity_text}'  # Varlığın başlangıç etiketi
                for i in range(token_start + 1, token_end + 1):
                    token_labels[i] = f'I-{entity_text}'  # Varlığın iç etiketi

        tokens.append(tokenized_text)
        labels.append(token_labels)

    return tokens, labels

# Metinleri tokenize et ve varlıkları etiketle
tokens, labels = tokenize_and_annotate(df)

# Token'lar ve etiketler ile bir DataFrame oluştur
annotated_df = pd.DataFrame({
    'Tokens': tokens,  # Token sütunu
    'Labels': labels  # Etiket sütunu
})

# DataFrame'i bir Excel dosyasına kaydet
output_excel_path = 'data.xlsx'
annotated_df.to_excel(output_excel_path, index=False)

print(f"Tokenized and annotated dataset has been saved to {output_excel_path}")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


JSON dosyası başarıyla oluşturuldu.
Tokenized and annotated dataset has been saved to data.xlsx


In [ ]:
import pandas as pd
from simpletransformers.ner import NERModel, NERArgs

# 1. Veriyi Yükle ve Hazırla
# Excel dosyasından veriyi yükler
df = pd.read_excel('/content/data.xlsx')

# String olarak kaydedilen liste temsillerini gerçek Python listelerine dönüştürür
df['Tokens'] = df['Tokens'].apply(eval)
df['Labels'] = df['Labels'].apply(eval)

# SimpleTransformers formatına uygun hale getirir
data = []
sentence_id = 0  # Cümle ID'si için başlangıç değeri
for tokens, labels in zip(df['Tokens'], df['Labels']):
    for token, label in zip(tokens, labels):
        data.append([sentence_id, token, label])  # Her bir token ve etiketi bir liste olarak ekler
    sentence_id += 1  # Bir sonraki cümle için cümle ID'sini artırır

# Hazırlanan veriyi bir DataFrame'e dönüştürür
data_df = pd.DataFrame(data, columns=['sentence_id', 'words', 'labels'])

# Tüm etiketlerin listesini çıkarır (benzersiz etiketler)
labels_list = list(set([label for labels in df['Labels'] for label in labels]))

# 2. Modeli Konfigüre Et ve Eğit
# Modelin eğitim ayarlarını yapar
model_args = NERArgs()
model_args.num_train_epochs = 10  # Eğitim epoch sayısını belirler
model_args.train_batch_size = 16  # Eğitim batch boyutunu belirler
model_args.eval_batch_size = 16  # Değerlendirme batch boyutunu belirler
model_args.learning_rate = 3e-5  # Öğrenme oranını belirler
model_args.save_steps = -1  # Adım başına kaydetmeyi devre dışı bırakır
model_args.save_model_every_epoch = False  # Her epoch sonunda modeli kaydetmeyi devre dışı bırakır
model_args.overwrite_output_dir = True  # Çıktı dizinini üzerine yazmak için ayar
model_args.labels_list = labels_list  # Etiket listesini model argümanlarına ekler

# BERT tabanlı bir Named Entity Recognition (NER) modelini oluşturur
model = NERModel(
    "bert",  # Model türü
    "dbmdz/bert-base-turkish-128k-uncased",  # Türkçe BERT modeli
    args=model_args,  # Eğitim ayarları
    use_cuda=True  # CUDA kullanarak eğitimi hızlandırmak için
)

# Modeli eğitim verisi üzerinde eğitir
model.train_model(data_df)

# 3. Modeli Değerlendir
# Eğitilmiş modeli veri seti üzerinde değerlendirir
result, model_outputs, preds_list = model.eval_model(data_df)
print(result)  # Değerlendirme sonuçlarını yazdırır

# 4. Yeni Cümleler Üzerinde Tahmin Yap
# Yeni bir cümle üzerinde modelin tahmin yapmasını sağlar
predictions, raw_outputs = model.predict(["Bu ürünü çok beğendim, ses kalitesi harika, paketlemesi çok kötüydü"])
print(predictions)  # Tahmin edilen varlıkları ve etiketleri yazdırır


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

# 1. CSV dosyasını oku
# Veriyi CSV dosyasından yükler
df = pd.read_csv("/content/sonHali.csv")

# 2. Veriyi uygun formata getir
data = []
for _, row in df.iterrows():
    # Her satır için entity listesi başlatılır
    entities = []

    # NaN (eksik) değilse, değeri string olarak alır
    if not pd.isna(row['duygu_label']):
        # JSON objesini yükler
        try:
            entities = json.loads(str(row['duygu_label']))
        except json.JSONDecodeError:
            # JSON hatası durumunda hatayı bildirir ve devam eder
            print(f"JSON decode error for row: {row['duygu_label']}")
            continue

    # Her entity üzerinde iterasyon yapar ve veriyi listeye ekler
    for entity in entities:
        # Entity geçerli bir dict ise ve 'text' ve 'labels' anahtarlarını içeriyorsa
        if isinstance(entity, dict) and 'text' in entity and 'labels' in entity:
            data.append({
                "text": entity["text"],
                "labels": entity["labels"][0]  # Bir etiket olduğunu varsayar
            })
        else:
            # Geçersiz entity durumunda hatayı bildirir
            print(f"Skipping invalid entity: {entity}")

# 3. Eğitim verisini DataFrame'e dönüştür
# Hazırlanan veriyi bir DataFrame'e dönüştürür
train_df = pd.DataFrame(data)

# 4. Veriyi eğitim ve doğrulama setlerine böl
# Veriyi %90 eğitim ve %10 doğrulama seti olarak böler
train_data, eval_data = train_test_split(train_df, test_size=0.1, random_state=42)

# 5. Etiketlerin listesini çıkar
# Etiketleri liste olarak tanımlar ve bunlara karşılık gelen numaraları mapler
labels_list = ["negative", "neutral", "positive"]
labels_map = {"negative": 0, "neutral": 1, "positive": 2}

# 6. Model argümanlarını belirle
# Model için gerekli olan ayarları yapar
args = ClassificationArgs()
args.num_train_epochs = 10  # Eğitim epoch sayısını belirler
args.learning_rate = 3e-5  # Öğrenme oranını belirler
args.overwrite_output_dir = True  # Çıktı dizinini üzerine yazmayı etkinleştirir
args.train_batch_size = 16  # Eğitim batch boyutunu belirler
args.eval_batch_size = 16  # Değerlendirme batch boyutunu belirler
args.save_steps = -1  # Adım başına kaydetmeyi devre dışı bırakır
args.save_model_every_epoch = False  # Her epoch sonunda kaydetmeyi devre dışı bırakır
args.labels_list = labels_list  # Etiket listesini model argümanlarına ekler
args.output_dir = "sentiment_model_output"  # Çıktı modelinin kaydedileceği dizin
args.labels_map = None  # Etiket map'ini None olarak ayarlar

# 7. BERT tabanlı duygu analizi modelini oluştur
# BERT modelini oluşturur, Türkçe diline özel eğitimli versiyonu kullanır
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-cased', args=args, num_labels=len(labels_list))

# 8. Modeli eğit
# Modeli eğitim verisi üzerinde eğitir ve doğrulama setini kullanarak değerlendirme yapar
model.train_model(train_data, eval_df=eval_data)

# 9. Modeli kaydet
# Eğitilmiş modeli belirtilen dizine kaydeder
model.save_model("/content/sentiment_model_output")

# 10. Metrik fonksiyonlarını tanımla
# Precision, recall ve f1 skorlarını hesaplamak için metrik fonksiyonlarını tanımlar
def compute_metrics(preds, labels):
    assert len(preds) == len(labels)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    return {
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }

# 11. Doğrulama sonuçlarını al ve metrikleri hesapla
# Doğrulama verisi üzerinde modelin performansını değerlendirir ve metrikleri hesaplar
result, model_outputs, wrong_predictions = model.eval_model(eval_data, compute_metrics=compute_metrics)

# Sonuçları yazdır
print("Validation Results:", result)


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

...

Running Epoch 10 of 10:   0%|          | 0/513 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/57 [00:00<?, ?it/s]

Validation Results: {'mcc': 0.5008351240121002, 'compute_metrics': {'precision': 0.8698939194131128, 'recall': 0.823271130625686, 'f1_score': 0.8409081647360647}, 'eval_loss': 0.518063962982412}


In [ ]:
# Google Drive'a dosya kopyalama işlemi
!cp /content/redmiBudsEssential_duzenlenmis.xlsx /content/drive/MyDrive

In [ ]:
from simpletransformers.ner import NERModel
from simpletransformers.classification import ClassificationModel

# 1. NER modelini yükle
ner_model = NERModel("bert", "/content/outputs", use_cuda=True)

# 2. Duygu analizi modelini yükle
sentiment_model = ClassificationModel("bert", "/content/sentiment_model_output", use_cuda=True)

# 3. Örnek cümle tanımla
sentence = "Bu ürünü çok beğendim, ses kalitesi harika, paketleme iyi değil"

# 4. Cümle üzerinde NER tahmini yap
ner_predictions, _ = ner_model.predict([sentence])

# 5. NER çıktısından varlıkları ve etiketlerini çıkar
entities = []
current_entity = []
current_label = None

# Her bir token üzerinde döngü yaparak varlıkları ve etiketleri topla
for token_info in ner_predictions[0]:
    for token, label in token_info.items():
        if label.startswith('B-') or (label.startswith('I-') and current_label != label[2:]):
            # Yeni bir varlık başlıyorsa, mevcut varlığı listeye ekle ve yeni varlığı başlat
            if current_entity:
                entities.append((" ".join(current_entity), current_label))
                current_entity = []
            current_entity.append(token)
            current_label = label[2:]
        elif label.startswith('I-') and current_label == label[2:]:
            # Mevcut varlığın devamıysa, token'ı mevcut varlığa ekle
            current_entity.append(token)
        else:
            # Mevcut varlık sona erdiyse, listeye ekle ve devam et
            if current_entity:
                entities.append((" ".join(current_entity), current_label))
                current_entity = []
            current_label = None

# Son varlığı ekle
if current_entity:
    entities.append((" ".join(current_entity), current_label))

# Cümleyi ve varlıkları yazdır
print(sentence)
print("Entities:")
for entity_text, label in entities:
    print(f"Entity: {entity_text}, Label: {label}")

# 6. Varlıklar üzerinde duygu analizi yap
entity_texts = [entity_text for entity_text, _ in entities]
sentiment_predictions, _ = sentiment_model.predict(entity_texts)

# 7. Sentiment predictions formatını kontrol et
print("Sentiment Predictions Format:")
print(sentiment_predictions)

# 8. Duygu analizi sonuçlarını yazdır
print("Sentiment Analysis:")
sentiment_labels = ["negative", "neutral", "positive"]
for entity_text, sentiment in zip(entity_texts, sentiment_predictions):
    print(f"Entity: {entity_text}, Sentiment: {sentiment}")


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

Bu ürünü çok beğendim, ses kalitesi harika, paketleme iyi değil
Entities:
Entity: ses kalitesi, Label: Ses kalitesi
Entity: paketleme, Label: Teslimat ve Paketleme
Entity: iyi, Label: Paketleme ve Teslimat


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Sentiment Predictions Format:
['positive', 'positive', 'positive']
Sentiment Analysis:
Entity: ses kalitesi, Sentiment: positive
Entity: paketleme, Sentiment: positive
Entity: iyi, Sentiment: positive


In [ ]:
import pandas as pd
from transformers import pipeline
import torch

# GPU kullanımı için cihazı ayarla (GPU varsa kullan, yoksa CPU kullan)
device = 0 if torch.cuda.is_available() else -1

# Entity'leri tanımlama fonksiyonu
def identify_entities(text, nlp_model):
    entities = nlp_model(text)
    merged_entities = []
    current_entity = ""
    current_label = ""

    for entity in entities:
        word = entity['word'].replace('##', '')  # ## sembollerini kaldır
        if entity['entity'].startswith('B-'):  # B- ile başlayanlar yeni entity'lerdir
            if current_entity:
                merged_entities.append((current_entity.strip(), current_label.split('-')[1]))
            current_entity = word
            current_label = entity['entity']
        elif entity['entity'].startswith('I-') and current_entity:  # I- ile devam eden entity'ler
            current_entity += " " + word
        else:
            if current_entity:
                merged_entities.append((current_entity.strip(), current_label.split('-')[1]))
                current_entity = ""
                current_label = ""

    if current_entity:  # Son kalan entity'yi de ekle
        merged_entities.append((current_entity.strip(), current_label.split('-')[1]))

    return merged_entities

# Duygu analizini gerçekleştiren fonksiyon
def analyze_sentiment(text, nlp_model):
    if not text:  # Eğer metin yoksa nötr olarak döndür
        return "neutral"
    sentiment = nlp_model(text)
    label_map = {
        "LABEL_2": "positive",  # Modelin etiketlerini daha anlaşılır hale getir
        "LABEL_0": "negative",
        "LABEL_1": "neutral"
    }
    return label_map.get(sentiment[0]['label'], "neutral")

# Yorumları okuma ve işleme fonksiyonu
def process_reviews(input_filename, entity_model_dir, sentiment_model_dir, output_filename):
    # xlsx dosyasından veri oku
    df = pd.read_excel(input_filename)

    # NaN değerlerini içeren satırları sil
    df.dropna(subset=["Yorum"], inplace=True)

    # NER ve Duygu Analizi modellerini yükle
    nlp_entity = pipeline("ner", model=entity_model_dir, tokenizer=entity_model_dir, device=device)
    nlp_sentiment = pipeline("sentiment-analysis", model=sentiment_model_dir, tokenizer=sentiment_model_dir, device=device)

    new_rows = []

    for i, row in df.iterrows():
        review_text = row["Yorum"]
        entities = identify_entities(review_text, nlp_entity)

        if entities:
            for entity, category in entities:
                sentiment = analyze_sentiment(entity, nlp_sentiment)
                new_row = row.copy()
                new_row["Entities"] = entity  # Tespit edilen entity'yi ekle
                new_row["Categories"] = category  # Entity'nin kategorisini ekle
                new_row["Sentiments"] = sentiment  # Entity'nin sentiment (duygu) etiketini ekle
                new_rows.append(new_row)
        else:
            new_row = row.copy()
            new_row["Entities"] = "yok"  # Eğer entity yoksa "yok" olarak ekle
            new_row["Categories"] = "yok"  # Kategori de "yok" olarak ekle
            new_row["Sentiments"] = "neutral"  # Duygu olarak "neutral" ekle
            new_rows.append(new_row)

    new_df = pd.DataFrame(new_rows)

    # 'Categories' sütunu "yok" olan satırları sil
    new_df = new_df[new_df["Categories"] != "yok"]

    # İşlenmiş veriyi yeni bir Excel dosyasına kaydet
    new_df.to_excel(output_filename, index=False)

# Dosya ve model dizinleri
input_xlsx = "/content/redmiBudsEssential.xlsx"  # Giriş dosyası
output_xlsx = 'redmiBudsEssential_duzenlenmis.xlsx'  # Çıkış dosyası
entity_model_dir = "/content/drive/MyDrive/ner"  # NER model dizini
sentiment_model_dir = "/content/drive/MyDrive/duygu"  # Duygu analizi model dizini

# Yorumları işle ve yeni dosyaya yaz
process_reviews(input_xlsx, entity_model_dir, sentiment_model_dir, output_xlsx)


In [ ]:
import pandas as pd
from transformers import pipeline

# Entity recognition modelini yükler
def load_entity_recognition_model(model_path):
    # Entity recognition modelini yükleme işlemi
    nlp = pipeline("ner", model=model_path, tokenizer=model_path)
    return nlp

# Entity'leri tanımlar ve kategorileri çıkartır
def identify_categories(text, nlp_model):
    # Cümledeki entity'leri tanımlama
    entities = nlp_model(text)
    print(f"Entity'ler tanımlandı: {entities}")  # Debugging line
    categories = set()
    for entity in entities:
        category = entity['entity'].split('-')[-1].lower()  # Kategori ismi
        categories.add(category)
    return list(categories)

# Kategorilere göre puan hesaplaması yapar
def calculate_scores(user_categories, model_data):
    # Kullanıcı kategorilerine göre puan hesaplama
    category_total_counts = {category: 0 for category in user_categories}
    positive_category_counts = {category: 0 for category in user_categories}
    print(f"Kullanıcı kategorileri: {user_categories}")  # Debugging line

    # Kategori sayımlarını yapar
    for index, row in model_data.iterrows():
        model_categories = row['Categories'].lower().split(', ')  # Case insensitive karşılaştırma
        model_sentiments = row['Sentiments'].lower().split(', ')
        like_count = row.get('Beğeni Sayısı', 0)  # Beğeni Sayısını al, eğer NaN ise 0 olarak kabul et

        for category, sentiment in zip(model_categories, model_sentiments):
            if category in user_categories:
                category_total_counts[category] += 1 + like_count  # 1 (Yorum yapan) + Beğeni Sayısı
                if sentiment == 'positive':  # Pozitif sentiment ise sayıyı artır
                    positive_category_counts[category] += 1 + like_count  # 1 (Yorum yapan) + Beğeni Sayısı

    # Puanları hesaplar
    category_scores = {}
    for category in user_categories:
        total = category_total_counts[category]
        if total > 0:
            score = (positive_category_counts[category] / total) * 100
            category_scores[category] = score

    print(f"Kategori sayıları: {category_total_counts}")  # Debugging line
    print(f"Pozitif kategori sayıları: {positive_category_counts}")  # Debugging line

    return category_scores

# XLSX dosyalarını işleme fonksiyonu
def process_xlsx_files(file_list, user_input):
    # XLSX dosyalarını işleme ve puanları hesaplama
    all_scores = {}
    for file_path in file_list:
        print(f"İşlenen dosya: {file_path}")

        # XLSX dosyasını yükler
        df = pd.read_excel(file_path)

        # NaN değerlerini içeren satırları siler
        df.dropna(subset=["Yorum"], inplace=True)

        # Kullanıcı girdisinden kategorileri tanımlar
        user_categories = identify_categories(user_input, nlp_model)
        if not user_categories:
            print(f"Hiçbir kategori tanımlanmadı: {user_input}")
            continue

        # Puanlamayı yapar
        category_scores = calculate_scores(user_categories, df)
        all_scores[file_path] = category_scores

    return all_scores

# Modeli yükler
entity_model_path = "/content/outputs"
nlp_model = load_entity_recognition_model(entity_model_path)

# Kullanıcı girdisini alır
user_input = "Kulaklık güzel olsun, ses kalitesi çok iyi, pil ömrü uzun"

# İşlenecek XLSX dosyalarının listesi
file_list = [
    "/content/jblTune500_duzenlenmis.xlsx",
    "/content/jblTune510bt_duzenlenmis.xlsx",
    "/content/pistonBasicEdition_duzenlenmis.xlsx",
    "/content/redmiBudsEssential_duzenlenmis.xlsx"
]

# XLSX dosyalarını işleyelim ve puanları hesaplar
scores = process_xlsx_files(file_list, user_input)

# Sonuçları ekrana yazdırır
print("Puanlama Sonuçları:")
for file_path, file_scores in scores.items():
    print(f"\nDosya Adı: {file_path}")
    for category, score in file_scores.items():
        print(f"    Kategori: {category}, Puan: {score:.2f}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


İşlenen dosya: /content/jblTune500_duzenlenmis.xlsx
Identified entities: [{'entity': 'B-Ses kalitesi', 'score': 0.9920964, 'index': 5, 'word': 'ses', 'start': 22, 'end': 25}, {'entity': 'I-Ses kalitesi', 'score': 0.9962035, 'index': 6, 'word': 'kalitesi', 'start': 26, 'end': 34}, {'entity': 'I-Ses kalitesi', 'score': 0.5814143, 'index': 7, 'word': 'çok', 'start': 35, 'end': 38}, {'entity': 'B-Pil Ömrü', 'score': 0.9971132, 'index': 10, 'word': 'pil', 'start': 44, 'end': 47}, {'entity': 'I-Pil Ömrü', 'score': 0.9150779, 'index': 11, 'word': 'ömrü', 'start': 48, 'end': 52}]
Kullanıcı kategorileri: ['pil ömrü', 'ses kalitesi']
Kategori sayıları: {'pil ömrü': 5, 'ses kalitesi': 515}
Pozitif kategori sayıları: {'pil ömrü': 2, 'ses kalitesi': 485}
İşlenen dosya: /content/jblTune510bt_duzenlenmis.xlsx
Identified entities: [{'entity': 'B-Ses kalitesi', 'score': 0.9920964, 'index': 5, 'word': 'ses', 'start': 22, 'end': 25}, {'entity': 'I-Ses kalitesi', 'score': 0.9962035, 'index': 6, 'word': 'k